In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp, Pauli
import networkx as nx
import numpy as np
from scipy.optimize import minimize
from qiskit.primitives import Estimator

def qaoa_ansatz(num_qubits, p):
    graph = nx.complete_graph(num_qubits)
    qc = QuantumCircuit(num_qubits)
    beta = [Parameter(f'β{i}') for i in range(p)]
    gamma = [Parameter(f'γ{i}') for i in range(p)]
    qc.h(range(num_qubits))  # Initialize in |+>
    for i in range(p):
        for u, v in graph.edges:
            qc.rzz(2 * gamma[i], u, v)  # Cost Hamiltonian
        qc.rx(2 * beta[i], range(num_qubits))  # Mixer
    return qc, beta + gamma


def maxcut_hamiltonian(graph, num_qubits):
    """Generate the MaxCut Hamiltonian for a given graph."""
    paulis = []
    coeffs = []
    
    # Generate terms for the Hamiltonian
    for u, v in graph.edges:
        # Add -1/2 * Z_u Z_v term
        pauli_str = ['I'] * num_qubits
        pauli_str[u] = 'Z'
        pauli_str[v] = 'Z'
        paulis.append(Pauli("".join(pauli_str)))
        coeffs.append(0.5)
        
        #Add +1/2 term (identity operator)
        paulis.append(Pauli("I" * num_qubits))
        coeffs.append(-0.5)
    
    return SparsePauliOp(paulis, coeffs)


def cost_function(params, ansatz, hamiltonian, num_qubits, p):
    qc, theta = ansatz(num_qubits, p)
    
    # Create a dictionary to assign parameters to the circuit
    param_dict = {theta[i]: params[i] for i in range(len(theta))}
    
    # Assign parameters to the circuit
    bound_circuit = qc.assign_parameters(param_dict)
    
    # Create an estimator primitive
    estimator = Estimator()
    
    # Calculate the expectation value of the Hamiltonian
    result = estimator.run(bound_circuit, [hamiltonian])
    expectation_value = result.result().values[0]

    return expectation_value

def optimization_loop(hamiltonian, num_qubits, p, optimizer, initial_params):
    def cost(params):
        return cost_function(params, qaoa_ansatz, hamiltonian, num_qubits, p)
        #print(f"Params: {params}, Energy: {energy}")
        return energy
    
    result = minimize(cost, initial_params, method=optimizer, options={'maxiter': 1000, 'tol': 1e-6})
    
    return result.fun, result.x

# Define the number of qubits and layers (p)
num_qubits = 12 #number of qubits
p = 1 # Number of QAOA layers

# Define the graph for MaxCut
graph = nx.complete_graph(num_qubits)  # All-to-all graph

# Generate the MaxCut Hamiltonian
hamiltonian = maxcut_hamiltonian(graph, num_qubits)

# Generate initial parameters
initialization_params = np.random.uniform(0, 2 * np.pi, 2 * p)
#initialization_params = np.zeros(2 * p)  # Or try np.pi * np.ones(2 * p)
# Define the optimizer
optimizer = 'COBYLA'

# Perform the optimization manually
optimal_value, optimal_params = optimization_loop(hamiltonian, num_qubits, p, optimizer, initialization_params)
print(f'Optimal energy: {optimal_value}')